In [1]:
import pandas as pd
import apyori
df = pd.read_csv('data_fin.csv', sep=';')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100480507 entries, 0 to 100480506
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Cust_Id   int64  
 1   Rating    float64
 2   Movie_Id  int64  
dtypes: float64(1), int64(2)
memory usage: 2.2 GB


In [4]:
good = df[df['Rating']==5].groupby('Cust_Id')['Movie_Id'].apply(lambda r: ' '.join([str(A) for A in r]))

In [5]:
good.sample(5)

Cust_Id
1223226    175 313 494 658 788 798 985 1032 1163 1202 152...
2476125    1905 2452 3917 5582 9254 9628 10778 10820 1152...
817584     2782 7745 11283 12470 14313 14538 15062 15124 ...
549702     28 571 705 1180 1220 1324 1406 1495 1599 1625 ...
999638     113 175 191 223 457 789 872 1032 1102 1220 130...
Name: Movie_Id, dtype: object

In [17]:
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.04, 
                                   min_confidence=0.1, min_lift=2, 
                                   min_length=2)

In [18]:
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

In [19]:
asr_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2023 entries, 0 to 2022
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   from        2023 non-null   object
 1   to          2023 non-null   object
 2   confidence  2023 non-null   object
 3   support     2023 non-null   object
 4   lift        2023 non-null   object
dtypes: object(5)
memory usage: 174.8+ KB


In [9]:
asr_df.sample(5)

,from,to,confidence,support,lift
3,11521,2452,0.806733,0.16798,3.95134
5,11521,7230,0.501797,0.104485,4.03989
15,7057,7230,0.886679,0.110883,7.13852
10,14961,2452,0.822189,0.102509,4.02705
14,2452,7230,0.512218,0.104578,4.12379


In [20]:
titles = pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1", 
                     header = None, 
                     names = ['Movie_Id', 'Year', 'Name'])

In [21]:
def get_rule_title(rule):
    print(titles[titles.Movie_Id.isin(rule['from'].split(' '))]['Name'].values)
    print('----------->')
    print(titles[titles.Movie_Id == int(rule['to'])]['Name'].values)

In [26]:
get_rule_title(asr_df.iloc[315])

['The Patriot']
----------->
['The Green Mile']


In [23]:
good[130]

'1865 3456 3962 5515 6029 6428 8159 8327 8782 8784 11165 11242 11701 12084 12232 14482'

In [28]:
j = 21
titles[titles.Movie_Id.isin(good.iloc[j].split(' '))]['Name']

3937                         Shrek 2
6735                          Robots
10909    The Women of Brewster Place
16277        Something the Lord Made
Name: Name, dtype: object

In [29]:
def print_rule_title(rule):
    return (titles[titles.Movie_Id == int(rule['to'])]['Name'].values)
    

result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(' ')) & set(asr_df['from'].loc[A].split(' '))) == len(asr_df['from'].loc[A].split(' ')):
        result.append(print_rule_title(asr_df.loc[A])[0])
print(set(result))

{'The Lord of the Rings: The Fellowship of the Ring: Extended Edition', 'Star Wars: Episode V: The Empire Strikes Back', 'Lord of the Rings: The Two Towers: Extended Edition', 'Finding Nemo (Widescreen)', 'The Sixth Sense'}
